In [3]:
# one off operation
!pip install -q duckdb

StatementMeta(, ba01ca46-d6a5-47cc-b86c-7b12404dbd9d, 5, Finished, Available, Finished)

Register an App if running outside Fabric to get a token
https://fivetran.com/docs/destinations/onelake/setup-guide

In [6]:
import duckdb
def attach_lakehouse(workspace,LH):
    try:
      token = notebookutils.credentials.getToken('storage')
    except:
        !pip install -q azure-identity
        from   azure.identity   import ClientSecretCredential
        import configparser
        config = configparser.ConfigParser()
        config.read("C:/KV/variable.ini")
        credential = ClientSecretCredential(
                        client_id     =config.get("myvars", "appId"),
                        client_secret =config.get("myvars", "secret") , 
                        tenant_id     =config.get("myvars", "tenantId")
                        )
        token =       credential.get_token("https://storage.azure.com/.default").token
    duckdb.sql(f""" CREATE or replace SECRET secret4 ( TYPE AZURE, PROVIDER ACCESS_TOKEN, ACCESS_TOKEN '{token}') """)
    list_tables = duckdb.sql(f""" SELECT  distinct(split_part(file, '_delta_log', 1)) as tables FROM glob ("abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/*/*/_delta_log/00000000000000000000.json") """).df()['tables'].tolist()
    sql_schema = set()
    sql_statements = set()
    for table_path in list_tables:
        parts = table_path.strip("/").split("/")
        schema = parts[-2]
        table = parts[-1]
        sql_schema.add(f"CREATE SCHEMA IF NOT EXISTS {schema};")
        sql_statements.add(f"""CREATE OR REPLACE VIEW {schema}.{table} AS SELECT * 
                             FROM delta_scan('abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{LH}.Lakehouse/Tables/{schema}/{table}');""")
    duckdb.sql(" ".join(sql_schema))
    duckdb.sql(" ".join(sql_statements))
    display(duckdb.sql("SHOW ALL TABLES").df())
attach_lakehouse("sqlengines","ETL")

StatementMeta(, ba01ca46-d6a5-47cc-b86c-7b12404dbd9d, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 20d4c7e6-633c-42f2-86ff-a92d2d07fc14)